In [ ]:
#@title 

# !gdown 1recNSOzW-NTWizR2kchUrEjpDf_MmM0d # /content/geoTallyDF.csv • Updated
# !gdown 1QFliJ5kP60R3zY8A7_awSmRGsXB34Y5i # /content/citiesWithGeoData.json
# !gdown 1IcuSNjWzDwoCwgIULYU9g_Q24cnT4ez9 # /content/jzfinalDF.csv

def audioAlert(x=0):
  from google.colab import output

  if x==0 : output.eval_js('new Audio(url="https://upload.wikimedia.org/wikipedia/commons/b/b5/Mobile_phone_ring.ogg").play()')
  else    : output.eval_js('new Audio(url="https://upload.wikimedia.org/wikipedia/commons/3/34/Sound_Effect_-_Door_Bell.ogg").play()')


from os.path import exists

inputFiles = [
    ('1recNSOzW-NTWizR2kchUrEjpDf_MmM0d', 'geoTallyDF.csv'),
    ('1QFliJ5kP60R3zY8A7_awSmRGsXB34Y5i', 'citiesWithGeoData.json'),
    ('1IcuSNjWzDwoCwgIULYU9g_Q24cnT4ez9', 'jzfinalDF.csv')  
]

for f in inputFiles:
  if not exists(f[1]): 
    fileID = f[0]
    !gdown $fileID -q


def MAIN():
  print('RUNNING MAIN()')

  global initializedMAIN, df, data, data2, safeData
  initializedMAIN = 1

  import json
  import pandas as pd

  df = pd.read_csv('/content/jzfinalDF.csv')

  df['LOCATION']  = df['LOCATION'].apply(lambda x: ', '.join([x.split(',')[0].strip()   ,   x.split(',')[1].strip()])    if ',' in x else x )
  # df['CityState'] = df['CityState'].apply(  lambda x: x.split(',')[0].strip().capitalize() + ', ' + x.split(',')[1].strip().upper())

  f = open('citiesWithGeoData.json')
    
  data = json.load(f)

  data2 = data.copy()

  keepCities = []

  citySet = set()

  cleanLocs = [i.strip() for i in list(df['LOCATION'])]

  for feature in data["features"]:
      for entry in feature["properties"]:        
          cityState = (feature["properties"]["NAME"].strip().upper() + ', ' + feature["properties"]["ST".strip().upper()])
          
          citySet.add(cityState)
          
          if cityState in cleanLocs:
              keepCities.append(feature)
          
  uniqueList = []

  for i in keepCities:
    if not i in uniqueList:
      uniqueList.append(i)

  data2["features"] = uniqueList

  # safeData = data2.copy()

  # file_1 = open("file_1.json", "w")
  # json.dump(data2, file_1, indent = 6)
  # file_1.close()


  ### EXTRACT AND KEEP LARGEST POLYGON
  for idx,feature in enumerate(data2["features"]):
    if feature["geometry"]["type"] == 'MultiPolygon':
      temp = data2["features"][idx]["geometry"]["coordinates"][0]
      for polygon in data2["features"][idx]["geometry"]["coordinates"]:
        if len(polygon[0]) > len(temp[0]):
          temp = polygon
        data2["features"][idx]["geometry"]["coordinates"] = temp
        feature["geometry"]["type"] = 'Polygon'


  out_file = open("jzfile.json", "w")
  json.dump(data2, out_file, indent = 6)
  out_file.close()




def PLOT(reRunMAIN = 0):
  global df, initializedMAIN

  if reRunMAIN or 'initializedMAIN' not in globals() : 
    print('CALLING MAIN() FROM PLOT() \n')
    MAIN()
  else:
    print('MAIN() ALREADY RUN • DIRECT TO PLOT() \n')

  import plotly.express as px

  fig = px.choropleth_mapbox(df, 
                            geojson       = data2,  
                            color         = 'topic_4',
                            locations     = 'CityState',
                            featureidkey  = 'properties.CityState',
                            #  locations = 'GlobalID',
                            #  featureidkey = 'properties.GlobalID',
                            color_continuous_scale = "Viridis",
                            range_color   = (0, max(df['topic_4'])),
                            mapbox_style  = "carto-positron",
                            zoom          = 10, 
                            center        = {"lat": 33.770050, "lon": -118.193741},
                            opacity       = 0.5,
                            labels        = {'topic_4':'TOPIC 4'}
                            )


  fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

  fig.show()



### ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- 

if __name__ == '__main__':

  try:
    PLOT(reRunMAIN=1)
    audioAlert(0)
  except:
    audioAlert(1)




Output hidden; open in https://colab.research.google.com to view.